<a href="https://colab.research.google.com/github/pawan-cpu/Learn-Python-with-Pawan-Kumar/blob/main/Copy_of_2021_12_27_pawan_lesson37.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 37: Grouping & Aggregation


|Particulars|Description|
|-|-|
|**Topics Covered**|Univariate Grouping|
||The `agg()` Function|
||Multivariate Grouping and Aggregation|
|||
|**Lesson Description**|A student will learn to group a DataFrame to perform detailed analysis.|
|||
|**Lesson Duration**|45 minutes|
|||
|**Learning Outcomes**|Group the values about specific column(s)|
||Apply the aggregation operations on the `DataFrameGroupBy` objects|
|||

---

### Teacher-Student Tasks

In this class, we will learn grouping and aggregation. These are the most frequently used operations on a DataFrame as far as analysis is concerned. 

First, we will group the DataFrame by a particular set of values and then we will apply the aggregation operations such as count, sum, median, mean, standard deviation etc. 

Let's run the codes we covered in the previous classes and continue this session from **Task 1: Univariate Grouping** section.

---

### Recap

Run the code cell below to load the dataset and apply the operations on the DataFrame that were covered in the previous classes.

In [ ]:
# Run the code cell.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


# Loading the dataset.
csv_file = 'https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/air-quality/AirQualityUCI.csv'
df = pd.read_csv(csv_file, sep=';')

# Dropping the 'Unnamed: 15' & 'Unnamed: 16' columns.
df = df.drop(columns=['Unnamed: 15', 'Unnamed: 16'], axis=1) 

# Dropping the null values.
df = df.dropna()

# Creating a Pandas series containing 'datetime' objects.
dt_series = pd.Series(data = [item.split("/")[2] + "-" + item.split("/")[1] + "-" + item.split("/")[0] for item in df['Date']], index=df.index) + ' ' + pd.Series(data=[str(item).replace(".", ":") for item in df['Time']], index=df.index)
dt_series = pd.to_datetime(dt_series)

# Remove the Date & Time columns from the DataFrame and insert the 'dt_series' in it.
df = df.drop(columns=['Date', 'Time'], axis=1)
df.insert(loc=0, column='DateTime', value=dt_series)

# Get the Pandas series containing the year values as integers.
year_series = dt_series.dt.year

# Get the Pandas series containing the month values as integers.
month_series = dt_series.dt.month

# Get the Pandas series containing the day values as integers.
day_series = dt_series.dt.day

# Get the Pandas series containing the days of a week, i.e., Monday, Tuesday, Wednesday etc.
day_name_series = dt_series.dt.day_name()

# Add the 'Year', 'Month', 'Day' and 'Day Name' columns to the DataFrame.
df['Year'] = year_series
df['Month'] = month_series
df['Day'] = day_series
df['Day Name'] = day_name_series

# Sort the DataFrame by the 'DateTime' values in the ascending order. Also, display the first 10 rows of the DataFrame.
df = df.sort_values(by='DateTime')

# Create a function to replace the commas with periods in a Pandas series.
def comma_to_period(series):
    new_series = pd.Series(data=[float(str(item).replace(',', '.')) for item in series], index=df.index)
    return new_series

# Apply the 'comma_to_period()' function on the ''CO(GT)', 'C6H6(GT)', 'T', 'RH' and 'AH' columns.
cols_to_correct = ['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH'] # Create a list of column names.
for col in cols_to_correct: # Iterate through each column
    df[col] = comma_to_period(df[col]) # Replace the original column with the new series.

# Remove all the columns from the 'df' DataFrame containing more than 10% garbage value.
df = df.drop(columns=['NMHC(GT)', 'CO(GT)', 'NOx(GT)', 'NO2(GT)'], axis=1)

# Create a new DataFrame containing records for the years 2004 and 2005.
aq_2004_df = df[df['Year'] == 2004]
aq_2005_df = df[df['Year'] == 2005]

# Replace the -200 value with the median values for each column having indices between 1 and -4 (excluding -4) for the 2004 year DataFrame.
for col in aq_2004_df.columns[1:-4]:
  median = aq_2004_df.loc[aq_2004_df[col] != -200, col].median() # Get the median value for each column after excluding -200.
  aq_2004_df[col] = aq_2004_df[col].replace(to_replace=-200, value=median)

# Repeat the same exercise for the 2005 year DataFrame.
for col in aq_2005_df.columns[1:-4]:
  median = aq_2005_df.loc[aq_2005_df[col] != -200, col].median()
  aq_2005_df[col] = aq_2005_df[col].replace(to_replace=-200, value=median)

---

#### Task 1: Univariate Grouping

Before we begin grouping and aggregation, let's first get the list of columns we are left with after applying all the previous operations.

In [ ]:
# S1.1: Get the list of columns present in the DataFrame.
df.columns

Index(['DateTime', 'PT08.S1(CO)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)',
       'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH', 'Year', 'Month', 'Day',
       'Day Name'],
      dtype='object')

The description for all the columns containing data for air pollutants, temperature, relative humidity and absolute humidity is provided below.


|Columns|Description|
|-|-|
|PT08.S1(CO)|PT08.S1 (tin oxide) hourly averaged sensor response (nominally $\text{CO}$ targeted)|
|C6H6(GT)|True hourly averaged Benzene concentration in $\frac{\mu g}{m^3}$|
|PT08.S2(NMHC)|PT08.S2 (titania) hourly averaged sensor response (nominally $\text{NMHC}$ targeted)|
|PT08.S3(NOx)|PT08.S3 (tungsten oxide) hourly averaged sensor response (nominally $\text{NO}_x$ targeted)|
|PT08.S4(NO2)|PT08.S4 (tungsten oxide) hourly averaged sensor response (nominally $\text{NO}_2$ targeted)|
|PT08.S5(O3) |PT08.S5 (indium oxide) hourly averaged sensor response (nominally $\text{O}_3$ targeted)|
|T|Temperature in Â°C|
|RH|Relative Humidity (%)|
|AH|AH Absolute Humidity|

**The `groupby()` Function**

Now, let's group all the values by months, i.e., club all the values together for January, February, and so on.

To group the values of a DataFrame by a particular value, use the `groupby()` function. Inside the function, you have to pass the column name to `by` parameter to specify the column about which you want to group the values.

**Syntax of `groupby()` function:** `dataframe.groupby(by='column')`

You can additionally set the `sort` parameter either equal to `True` or `False` depending on whether you want to sort the values or not. By default, the `groupby()` function returns a new Pandas `DataFrameGroupBy` object containing the grouped sorted values:

In [ ]:
# S1.2: Group the records for the 2004 DataFrame together by month.
group_2004_month=aq_2004_df.groupby(by="Month")
group_2004_month

The `group_by()` function returns a `DataFrameGroupBy` object with its physical memory location.

How you want to group the values depends on the problem statement at hand. Here, we would like to analyze data month-wise. Hence, we have grouped the DataFrame by the `Month` column.

**The `get_group()` Function**

To get all the occurrences of a value from a group, use the `get_group()` function. 

**Syntax of `get_group()` function:** `DataFrameGroupBy_object.get_group(item)`

E.g., if you want to get all the records of March from the `group_2004_month DataFrameGroupBy` object, pass `3` as input to the `get_group()` function.

In [ ]:
# S1.3: Get all the records for the month of March.
group_2004_month.get_group(3)

,DateTime,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day,Day Name
0,2004-03-10 18:00:00,1360.0,11.9,1046.0,1056.0,1692.0,1268.0,13.6,48.9,0.7578,2004,3,10,Wednesday
1,2004-03-10 19:00:00,1292.0,9.4,955.0,1174.0,1559.0,972.0,13.3,47.7,0.7255,2004,3,10,Wednesday
2,2004-03-10 20:00:00,1402.0,9.0,939.0,1140.0,1555.0,1074.0,11.9,54.0,0.7502,2004,3,10,Wednesday
3,2004-03-10 21:00:00,1376.0,9.2,948.0,1092.0,1584.0,1203.0,11.0,60.0,0.7867,2004,3,10,Wednesday
4,2004-03-10 22:00:00,1272.0,6.5,836.0,1205.0,1490.0,1110.0,11.2,59.6,0.7888,2004,3,10,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,2004-03-31 19:00:00,1254.0,11.5,1033.0,819.0,1669.0,1068.0,14.3,52.9,0.8581,2004,3,31,Wednesday
506,2004-03-31 20:00:00,1198.0,9.3,953.0,882.0,1588.0,1019.0,13.2,56.0,0.8446,2004,3,31,Wednesday
507,2004-03-31 21:00:00,1060.0,5.5,787.0,1044.0,1418.0,833.0,12.6,57.0,0.8315,2004,3,31,Wednesday
508,2004-03-31 22:00:00,1050.0,5.1,769.0,1078.0,1409.0,759.0,12.3,57.8,0.8224,2004,3,31,Wednesday


The above DataFrame contains all the records only for March 2004.

In [ ]:
# S1.4: Get all the records for the month of April.
group_2004_month.get_group(4)

,DateTime,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day,Day Name
510,2004-04-01 00:00:00,1143.0,6.3,825.0,986.0,1477.0,978.0,12.0,61.6,0.8593,2004,4,1,Thursday
511,2004-04-01 01:00:00,1044.0,5.1,770.0,1031.0,1425.0,944.0,11.5,63.9,0.8652,2004,4,1,Thursday
512,2004-04-01 02:00:00,1034.0,4.1,716.0,1085.0,1405.0,891.0,10.7,67.2,0.8630,2004,4,1,Thursday
513,2004-04-01 03:00:00,956.0,4.0,713.0,1099.0,1422.0,849.0,9.0,73.1,0.8394,2004,4,1,Thursday
514,2004-04-01 04:00:00,909.0,2.4,615.0,1237.0,1322.0,790.0,10.2,66.6,0.8299,2004,4,1,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2004-04-30 19:00:00,1626.0,26.9,1476.0,555.0,2356.0,1786.0,19.8,58.6,1.3392,2004,4,30,Friday
1226,2004-04-30 20:00:00,1449.0,19.5,1282.0,625.0,2100.0,1569.0,19.1,61.1,1.3345,2004,4,30,Friday
1227,2004-04-30 21:00:00,1363.0,15.1,1152.0,684.0,1951.0,1495.0,18.2,65.4,1.3529,2004,4,30,Friday
1228,2004-04-30 22:00:00,1371.0,14.6,1136.0,689.0,1927.0,1471.0,18.1,66.1,1.3579,2004,4,30,Friday


The above DataFrame contains all the records only for April 2004.

Similarly, you can get values for all other months from the `grouped_2004_month DataFrameGroupBy` object.

Now, you group the records of 2005 about the `Month` column so that we can later compare the situation in 2004 with 2005.

In [ ]:
# S1.5: Group the records for the 2005 DataFrame together by month.
group_2005_month=aq_2005_df.groupby(by="Month")
group_2005_month

---

#### Task 2: Aggregation on Univariate Grouping

You have already learnt the aggregation functions such as `count(), sum(), min(), max(), mean()` etc. Let's apply a few of them on the `DataFrameGroupBy` objects that we have already created.

The pollution is most severe during the winter season in any part of the world. The winter season in Italy begins from the last week of October and lasts till the last week of March. So, let's get the descriptive statistics for the concentrations of the air pollutants, temperature, relative humidity and absolute humidity in March, November and December in 2004 and 2005.

In [ ]:
# S2.1: Get the descriptive statistics for March 2004.
group_2004_month.get_group(3).describe()

,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day
count,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.0,510.0,510.000000
mean,1222.819608,9.932745,935.643137,1029.172549,1572.072549,1027.494118,14.390784,50.170784,0.789413,2004.0,3.0,20.870588
std,241.038541,7.060527,258.997406,268.130898,280.525357,381.600387,4.447539,14.265312,0.139932,0.0,0.0,6.141338
min,818.000000,0.600000,457.000000,537.000000,1050.000000,341.000000,6.100000,14.900000,0.402300,2004.0,3.0,10.000000
25%,1027.250000,4.600000,742.250000,834.250000,1359.000000,762.500000,11.000000,38.550000,0.691175,2004.0,3.0,16.000000
50%,1197.000000,8.650000,926.000000,966.500000,1543.000000,992.500000,14.050000,52.200000,0.792050,2004.0,3.0,21.000000
75%,1383.000000,13.650000,1106.250000,1175.500000,1730.000000,1272.000000,16.800000,60.750000,0.895900,2004.0,3.0,26.000000
max,2040.000000,39.200000,1754.000000,1935.000000,2679.000000,2359.000000,29.300000,83.200000,1.094500,2004.0,3.0,31.000000


In [ ]:
# S2.2: Get the descriptive statistics for March 2005.
group_2005_month.get_group(3).describe()

,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day
count,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.0,744.0,744.000000
mean,1145.223118,8.726478,887.740591,709.447581,1225.188172,1083.598118,13.213306,50.909677,0.781531,2005.0,3.0,16.000000
std,197.140374,6.677222,253.315169,216.393926,275.936945,408.951492,6.394258,16.598389,0.291686,0.0,0.0,8.950289
min,715.000000,0.200000,387.000000,330.000000,551.000000,221.000000,-1.900000,13.500000,0.184700,2005.0,3.0,1.000000
25%,996.000000,3.700000,692.500000,551.750000,1054.750000,808.500000,8.675000,37.400000,0.591475,2005.0,3.0,8.000000
50%,1126.000000,6.600000,841.000000,684.500000,1217.000000,1039.500000,13.500000,51.800000,0.783400,2005.0,3.0,16.000000
75%,1267.500000,12.100000,1052.500000,830.000000,1376.000000,1365.250000,17.600000,64.575000,0.991450,2005.0,3.0,24.000000
max,1818.000000,35.500000,1675.000000,1804.000000,2147.000000,2159.000000,28.800000,84.000000,1.393000,2005.0,3.0,31.000000


If we keep on aggregating the values month-wise one-by-one, then it will become a cumbersome task. Let's get the descriptive statistics for all the months for both the years 2004 and 2005.

In [ ]:
# S2.3: Get the descriptive statistics for all the months for the year 2004.
group_2004_month.describe()

PT08.S1(CO)                                  ...   Day                  
            count         mean         std    min  ...   25%   50%   75%   max
Month                                              ...                        
3           510.0  1222.819608  241.038541  818.0  ...  16.0  21.0  26.0  31.0
4           720.0  1158.993056  227.826933  753.0  ...   8.0  15.5  23.0  30.0
5           744.0  1076.350806  193.766317  736.0  ...   8.0  16.0  24.0  31.0
6           720.0  1022.862500  157.922850  708.0  ...   8.0  15.5  23.0  30.0
7           744.0  1046.452957  158.314118  723.0  ...   8.0  16.0  24.0  31.0
8           744.0   986.259409  117.964334  711.0  ...   8.0  16.0  24.0  31.0
9           720.0  1081.022222  197.420922  718.0  ...   8.0  15.5  23.0  30.0
10          744.0  1184.745968  243.017236  704.0  ...   8.0  16.0  24.0  31.0
11          720.0  1132.148611  273.058383  647.0  ...   8.0  15.5  23.0  30.0
12          744.0  1084.196237  211.089768  691.0  ...   8.0  16.0  24.0  31.0

[10 rows x 88 columns]

In [ ]:
# S2.4: Get the descriptive statistics for all the months for the year 2005.
group_2005_month.describe()

PT08.S1(CO)                                  ...   Day                   
            count         mean         std    min  ...   25%   50%    75%   max
Month                                              ...                         
1           744.0  1108.086022  214.147270  744.0  ...  8.00  16.0  24.00  31.0
2           672.0  1086.369048  184.077312  755.0  ...  7.75  14.5  21.25  28.0
3           744.0  1145.223118  197.140374  715.0  ...  8.00  16.0  24.00  31.0
4            87.0   953.712644  125.441352  811.0  ...  1.00   2.0   3.00   4.0

[4 rows x 88 columns]

We still have quite large DataFrames containing the descriptive statistics for all the months and all the columns. Let's try to retrieve only the mean, standard deviation and median values for all the months.

---

#### Task 3: The `agg()` Function

Instead of applying the aggregation functions on a `DataFrameGroupBy` object individually, you can apply them together using the `agg()` function. Inside the `agg()` function, you have to pass the names of the functions (as string values) as inputs.

**Syntax of `agg()` function:** `DataFrameGroupBy.agg(func=('func1', 'func2' ... 'funcN'))`

Where `'func1', 'func2' ... 'funcN'` are the names of the aggregation functions to be applied. They are passed as a singular value (through tuple) to the `func` parameter.

Let's apply the `mean(), std()` and `median()` function on the `group_2004_month DataFrameGroupBy` object:

In [ ]:
# S3.1: Get mean, standard deviation and median for all the months for the year 2004.
group_2004_month.agg(func=('mean','std','median'))


PT08.S1(CO)                      ...        Day                 
              mean         std  median  ...       mean       std median
Month                                   ...                            
3      1222.819608  241.038541  1197.0  ...  20.870588  6.141338   21.0
4      1158.993056  227.826933  1115.5  ...  15.500000  8.661458   15.5
5      1076.350806  193.766317  1053.0  ...  16.000000  8.950289   16.0
6      1022.862500  157.922850  1030.5  ...  15.500000  8.661458   15.5
7      1046.452957  158.314118  1032.0  ...  16.000000  8.950289   16.0
8       986.259409  117.964334   987.5  ...  16.000000  8.950289   16.0
9      1081.022222  197.420922  1060.0  ...  15.500000  8.661458   15.5
10     1184.745968  243.017236  1159.5  ...  16.000000  8.950289   16.0
11     1132.148611  273.058383  1106.5  ...  15.500000  8.661458   15.5
12     1084.196237  211.089768  1060.0  ...  16.000000  8.950289   16.0

[10 rows x 33 columns]

You can also provide the name of the functions to be applied as items in a list.

In [ ]:
# S3.2: Get mean, standard deviation and median for all the months for the year 2005.
group_2005_month.agg(func=('mean','std','median'))

PT08.S1(CO)                      ...        Day                 
              mean         std  median  ...       mean       std median
Month                                   ...                            
1      1108.086022  214.147270  1076.0  ...  16.000000  8.950289   16.0
2      1086.369048  184.077312  1070.0  ...  14.500000  8.083764   14.5
3      1145.223118  197.140374  1126.0  ...  16.000000  8.950289   16.0
4       953.712644  125.441352   925.0  ...   2.344828  1.065610    2.0

[4 rows x 33 columns]

**Note:** You also choose to apply different aggregation functions on different columns by passing a dictionary containing the names of the columns and the aggregation function(s) to be applied on them.

E.g., let's apply the `mean(), std()` & `median()` functions on the `T` (temperature) column and `max()` function on the `AH` (Absolute humidity) column.

In [ ]:
# S3.3: On 'group_2004_month', apply the 'mean', 'std' & 'median' functions on the 'T' column and 'max' function on the 'AH' column.
# Also apply the 'count' function on the 'Month' column.
group_2004_month.agg(func={'T': ['mean', 'std', 'median'], 'AH' : 'max', 'Month' : 'count'})


T                       AH Month
            mean       std median     max count
Month                                          
3      14.390784  4.447539  14.05  1.0945   510
4      16.935694  4.758767  16.30  1.4852   720
5      20.283468  5.163229  19.30  1.6296   744
6      26.155833  5.955815  24.70  1.9390   720
7      29.406989  6.178302  28.05  2.0042   744
8      28.555376  5.469489  27.55  2.1806   744
9      24.514861  5.440697  23.60  2.2310   720
10     20.497446  3.586663  20.30  2.0224   744
11     13.485417  4.990447  13.25  1.9800   720
12     12.136425  4.462879  12.00  1.3713   744

The least number of observations were recorded in March for the year 2004.

In [ ]:
# S3.4: On the 'group_2005_month', apply the 'min' & 'max' functions on the 'AH' and 'RH' columns. 
# Also apply the 'count' function on the 'Month' column.
group_2005_month.agg(func={'AH': ['min', 'max'], 'RH' : ['min', 'max'], 'Month' : 'count'})


AH            RH       Month
          min     max   min   max count
Month                                  
1      0.2477  1.0567  17.2  86.6   744
2      0.2269  1.0859  18.0  86.6   672
3      0.1847  1.3930  13.5  84.0   744
4      0.3866  0.8642   9.9  63.1    87

The least number of observations were recorded in April for the year 2005.

---

#### Task 4: Slicing `DataFrameGroupBy` Object


We still want to know the mean, standard deviation and median values only for the winter season for 2004 and 2005. 

So, we can use the `loc[]` function to get the rows only for the winter season.

In [ ]:
# S4.1: Get mean, standard deviation and median values for the winter season of 2004.
group_2004_month.agg(func=('mean','std','median')).loc[[3,11,12],:]

PT08.S1(CO)                      ...        Day                 
              mean         std  median  ...       mean       std median
Month                                   ...                            
3      1222.819608  241.038541  1197.0  ...  20.870588  6.141338   21.0
11     1132.148611  273.058383  1106.5  ...  15.500000  8.661458   15.5
12     1084.196237  211.089768  1060.0  ...  16.000000  8.950289   16.0

[3 rows x 33 columns]

But we don't want any of the aggregated values for the `Year` and `Day` columns. Hence, we need to specify the column indices for the columns we want to be displayed.

In [ ]:
# S4.2: Get the column indices for the 'group_2004_month.agg(func=['mean', 'std', 'median'])' DataFrame columns.
group_2004_month.agg(func=('mean','std','median')).loc[[3,11,12],:].columns

MultiIndex([(  'PT08.S1(CO)',   'mean'),
            (  'PT08.S1(CO)',    'std'),
            (  'PT08.S1(CO)', 'median'),
            (     'C6H6(GT)',   'mean'),
            (     'C6H6(GT)',    'std'),
            (     'C6H6(GT)', 'median'),
            ('PT08.S2(NMHC)',   'mean'),
            ('PT08.S2(NMHC)',    'std'),
            ('PT08.S2(NMHC)', 'median'),
            ( 'PT08.S3(NOx)',   'mean'),
            ( 'PT08.S3(NOx)',    'std'),
            ( 'PT08.S3(NOx)', 'median'),
            ( 'PT08.S4(NO2)',   'mean'),
            ( 'PT08.S4(NO2)',    'std'),
            ( 'PT08.S4(NO2)', 'median'),
            (  'PT08.S5(O3)',   'mean'),
            (  'PT08.S5(O3)',    'std'),
            (  'PT08.S5(O3)', 'median'),
            (            'T',   'mean'),
            (            'T',    'std'),
            (            'T', 'median'),
            (           'RH',   'mean'),
            (           'RH',    'std'),
            (           'RH', 'median'),
            (   

Here, we have a collection of column names. For each column, the column itself and the subcolumns (`mean, std, median`) are put together in a tuple. So, for every column, we have three tuples. 

We don't want the last six combinations of columns and sub columns to be part of our final aggregated DataFrame. Hence, we will pass the remaining column indices to the `loc[]` function.

In [ ]:
# S4.3: Get mean, standard deviation and median values for the winter season of 2004 without the 'Year' & 'Day' columns.
group_2004_month.agg(func=['mean', 'std', 'median']).loc[[3, 11, 12], group_2004_month.agg(func=['mean', 'std', 'median']).columns[:-6]]


PT08.S1(CO)                      ...        AH                   
              mean         std  median  ...      mean       std   median
Month                                   ...                             
3      1222.819608  241.038541  1197.0  ...  0.789413  0.139932  0.79205
11     1132.148611  273.058383  1106.5  ...  0.939991  0.360286  0.95040
12     1084.196237  211.089768  1060.0  ...  0.814761  0.291196  0.85780

[3 rows x 27 columns]

The `group_2004_month.agg(func=['mean', 'std', 'median']).columns[:-6]` is a list of combinations of all the columns and sub columns except for the last 6 combinations.

In [ ]:
# S4.4: Get mean, standard deviation and median values for the winter season of 2005 without the 'Year' & 'Day' columns.
group_2005_month.agg(func=['mean', 'std', 'median']).loc[[1,2,3], group_2005_month.agg(func=['mean', 'std', 'median']).columns[:-6]]

PT08.S1(CO)                      ...        AH                  
              mean         std  median  ...      mean       std  median
Month                                   ...                            
1      1108.086022  214.147270  1076.0  ...  0.634713  0.218392  0.6304
2      1086.369048  184.077312  1070.0  ...  0.530231  0.144846  0.5208
3      1145.223118  197.140374  1126.0  ...  0.781531  0.291686  0.7834

[3 rows x 27 columns]

**Note:** Unfortunately, the `quantile()` function cannot be applied as an aggregation function on a `DataFrameGroupBy` object. To get quartiles, use the `describe()` function.



In [ ]:
# S4.5: Get the list of all the columns of the 'aq_2004_df' DataFrame.
group_2004_month.describe().columns

MultiIndex([(  'PT08.S1(CO)', 'count'),
            (  'PT08.S1(CO)',  'mean'),
            (  'PT08.S1(CO)',   'std'),
            (  'PT08.S1(CO)',   'min'),
            (  'PT08.S1(CO)',   '25%'),
            (  'PT08.S1(CO)',   '50%'),
            (  'PT08.S1(CO)',   '75%'),
            (  'PT08.S1(CO)',   'max'),
            (     'C6H6(GT)', 'count'),
            (     'C6H6(GT)',  'mean'),
            (     'C6H6(GT)',   'std'),
            (     'C6H6(GT)',   'min'),
            (     'C6H6(GT)',   '25%'),
            (     'C6H6(GT)',   '50%'),
            (     'C6H6(GT)',   '75%'),
            (     'C6H6(GT)',   'max'),
            ('PT08.S2(NMHC)', 'count'),
            ('PT08.S2(NMHC)',  'mean'),
            ('PT08.S2(NMHC)',   'std'),
            ('PT08.S2(NMHC)',   'min'),
            ('PT08.S2(NMHC)',   '25%'),
            ('PT08.S2(NMHC)',   '50%'),
            ('PT08.S2(NMHC)',   '75%'),
            ('PT08.S2(NMHC)',   'max'),
            ( 'PT08.S3(NOx)', 'count'),


Now, let's have a look at how we can get only quartiles using the `describe()` function:

In [ ]:
# S4.6: For winters of 2004, get the quartiles of all the air pollutants, temperature, relative humidity and absolute humidity.
for col in aq_2004_df.columns[1:-4]:
  print(group_2004_month.describe().loc[[3, 11, 12], [(col, '25%'), (col, '50%'), (col, '75%')]], '\n')

      PT08.S1(CO)                
              25%     50%     75%
Month                            
3         1027.25  1197.0  1383.0
11         912.75  1106.5  1311.0
12         927.75  1060.0  1199.0 

      C6H6(GT)              
           25%    50%    75%
Month                       
3        4.600   8.65  13.65
11       5.000  10.10  17.60
12       4.375   8.60  12.50 

      PT08.S2(NMHC)                
                25%    50%      75%
Month                              
3            742.25  926.0  1106.25
11           762.50  981.0  1227.25
12           730.75  926.0  1067.00 

      PT08.S3(NOx)               
               25%    50%     75%
Month                            
3           834.25  966.5  1175.5
11          558.75  722.5   923.0
12          671.75  823.0   981.5 

      PT08.S4(NO2)                 
               25%     50%      75%
Month                              
3          1359.00  1543.0  1730.00
11         1111.00  1313.5  1597.50
12         104

First, we have applied the `describe()` function which will return all the descriptive statistics for the `DataFrameGroupBy` object. Then using the `loc[]` function we have retrieved only the quartile values for the winter season of 2004.

In [ ]:
# S4.7: For winters of 2005, get the quartiles of all the air pollutants, temperature, relative humidity and absolute humidity.
for col in aq_2005_df.columns[1:-4]:
  print(group_2005_month.describe().loc[[1,2,3], [(col, '25%'), (col, '50%'), (col, '75%')]], '\n')

      PT08.S1(CO)                 
              25%     50%      75%
Month                             
1          933.50  1076.0  1256.00
2          958.75  1070.0  1155.25
3          996.00  1126.0  1267.50 

      C6H6(GT)           
           25%  50%   75%
Month                    
1          3.6  6.3  13.1
2          3.6  6.3   9.5
3          3.7  6.6  12.1 

      PT08.S2(NMHC)                
                25%    50%      75%
Month                              
1            685.75  826.0  1087.25
2            688.75  826.0   960.00
3            692.50  841.0  1052.50 

      PT08.S3(NOx)               
               25%    50%     75%
Month                            
1           590.75  740.0  932.00
2           656.75  740.0  897.25
3           551.75  684.5  830.00 

      PT08.S4(NO2)                 
               25%     50%      75%
Month                              
1           948.00  1109.0  1319.75
2           917.00  1039.0  1160.25
3          1054.75  1217.0

**Note:** Since we have grouped the DataFrames for the years 2004 and 2005 by the `Month` column, it becomes the default index column in the aggregated DataFrames. If you don't want it to happen, pass the `as_index=False` parameter inside the `groupby()` function. 

In [ ]:
# S4.8: Group the DataFrame for the year 2004 by the 'Month' column without making it a default index column in the aggregated DataFrames.
# Apply the 'mean' function on the 'DataFrameGroupBy' object.
aq_2004_df.groupby(by="Month",as_index=False).mean()


,Month,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Day
0,3,1222.819608,9.932745,935.643137,1029.172549,1572.072549,1027.494118,14.390784,50.170784,0.789413,2004.0,20.870588
1,4,1158.993056,10.329583,948.565278,931.405556,1608.054167,1001.987500,16.935694,50.627569,0.927448,2004.0,15.500000
2,5,1076.350806,10.192339,950.735215,945.151882,1599.932796,928.598118,20.283468,43.636559,0.959994,2004.0,16.000000
3,6,1022.862500,10.500972,964.565278,896.379167,1712.698611,938.212500,26.155833,40.309861,1.266057,2004.0,15.500000
4,7,1046.452957,10.597043,971.427419,804.911290,1643.682796,994.391129,29.406989,33.075067,1.242517,2004.0,16.000000
5,8,986.259409,7.112634,844.643817,835.005376,1578.178763,788.688172,28.555376,43.346035,1.585142,2004.0,16.000000
6,9,1081.022222,11.485694,991.970833,810.725000,1546.534722,1034.245833,24.514861,44.105833,1.280758,2004.0,15.500000
7,10,1184.745968,13.518952,1058.263441,688.321237,1636.818548,1163.561828,20.497446,61.892944,1.463595,2004.0,16.000000
8,11,1132.148611,12.511806,1011.201389,789.880556,1372.234722,1171.354167,13.485417,59.274861,0.939991,2004.0,15.500000
9,12,1084.196237,9.423656,916.896505,876.579301,1251.625000,1068.323925,12.136425,56.825941,0.814761,2004.0,16.000000


As you can see, the `Month` column is not an index column of the mean aggregated DataFrame.

---

#### Task 5: Multivariate Grouping & Aggregation

We can also group multiple columns at once by passing a list of columns to be grouped.

Let's group the DataFrames about the `Month` & `Day Name` columns.

In [ ]:
# S5.1: Group the DataFrame for the year 2004 about the 'Month' & 'Day Name' columns.
aq_2004_df.groupby(by=["Month","Day Name"])

Let's get the count of each day for each month in 2004.


In [ ]:
# S5.2 Get the count of each day for each month in 2004.
aq_2004_df.groupby(by=["Month","Day Name"]).count()

DateTime  PT08.S1(CO)  C6H6(GT)  ...   AH  Year  Day
Month Day Name                                    ...                
3     Friday           72           72        72  ...   72    72   72
      Monday           72           72        72  ...   72    72   72
      Saturday         72           72        72  ...   72    72   72
      Sunday           72           72        72  ...   72    72   72
      Thursday         72           72        72  ...   72    72   72
...                   ...          ...       ...  ...  ...   ...  ...
12    Saturday         96           96        96  ...   96    96   96
      Sunday           96           96        96  ...   96    96   96
      Thursday        120          120       120  ...  120   120  120
      Tuesday          96           96        96  ...   96    96   96
      Wednesday       120          120       120  ...  120   120  120

[70 rows x 12 columns]

Let's get all the records for all Mondays in the month of March, 2004. 

**Note:** The values to be retrieved should be passed as a tuple in the same order of grouping.

In [ ]:
# S5.3: Get all the records for all Mondays in the month of March, 2004.
aq_2004_df.groupby(by=["Month","Day Name"]).get_group((3,"Monday"))

,DateTime,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day,Day Name
102,2004-03-15 00:00:00,1239.0,7.4,872.0,985.0,1547.0,1250.0,14.8,54.7,0.9164,2004,3,15,Monday
103,2004-03-15 01:00:00,1239.0,6.9,853.0,1010.0,1543.0,1174.0,14.0,57.0,0.9094,2004,3,15,Monday
104,2004-03-15 02:00:00,1224.0,7.0,855.0,998.0,1566.0,1149.0,13.4,61.3,0.9361,2004,3,15,Monday
105,2004-03-15 03:00:00,1078.0,4.4,734.0,1128.0,1487.0,1021.0,12.6,63.5,0.9230,2004,3,15,Monday
106,2004-03-15 04:00:00,1078.0,4.0,711.0,1150.0,1468.0,1013.0,12.3,65.4,0.9351,2004,3,15,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,2004-03-29 19:00:00,1127.0,8.2,907.0,957.0,1404.0,796.0,14.5,35.4,0.5839,2004,3,29,Monday
458,2004-03-29 20:00:00,1020.0,5.7,797.0,1066.0,1302.0,689.0,13.2,37.9,0.5733,2004,3,29,Monday
459,2004-03-29 21:00:00,949.0,3.8,700.0,1221.0,1178.0,553.0,13.1,34.4,0.5170,2004,3,29,Monday
460,2004-03-29 22:00:00,930.0,4.0,710.0,1278.0,1137.0,509.0,13.5,28.5,0.4405,2004,3,29,Monday


Similarly, you can retrieve records for different days in different months in a year.

Let's pause here. In the next class, we will continue with grouping, aggregation and learn to create time series plots. We will also learn how to customise the `matplotlib` plots.


---